In [1]:
import numpy as np
from pprint import pprint

np.set_printoptions(precision=6)

In [15]:
def get_x(mu, Theta):
    x = np.array([1/len(mu) for _ in range(len(mu))])
    for _ in range(100000):
        x = (mu*x).dot(Theta) / mu
    return x

def get_a(L, Q, k):
    a = [[0] * L for _ in range(Q + 1)]
    a[0] = a[1] = [1] * L
    for i in range(2, Q + 1):
        for j in range(L):
            a[i][j] = min(i, k[j])
    return a

def get_A(L, Q, a):
    A = [[1] * L for _ in range(Q + 1)]
    for i in range(Q + 1):
        for j in range(L):
            if i == 0:
                A[i][j] = 1
            else:
                # trouble ?
                A[i][j] *= a[i][j]
    return A

def get_G(L, Q, A, x):
    G = [[0] * L for _ in range((Q + 1))]
    for i in range(Q + 1):
        G[i][0] = x[0]**i / A[i][0]
    G[0] = [1] * L    
    
    for y in range(1, Q + 1):
        for z in range(1, L):
            for m in range(y + 1):
                G[y][z] += (x[z]**m / A[m][z]) * G[y - m][z - 1] 
    return G

In [25]:
# Вариант 2
# Тест
L = 6
Q = 10
k = [3, 2, 2, 2, 5, 4]
mu = np.array([.2, .2, .3, .3, .4, .2])
Theta = np.array([
    [.0, .4, .0, .0, .0, .6],
    [.2, .0, .7, .0, .1, .0],
    [.6, .0, .0, .4, .0, .0],
    [.0, .5, .0, .0, .5, .0],
    [.0, .0, .7, .0, .0, .3],
    [.0, .0, .1, .8, .1, .0]
])


# Решение системы линейных уравнений
x = get_x(mu, Theta)

# Вычисление базовых функций

# Коэффициенты интенсивности обслуживания
a = get_a(L, Q, k)
# Вспомогательная функция
A = get_A(L, Q, a)
# Базовые функции
G = get_G(L, Q, A, x)

G_m = [[0] * L for _ in range((Q + 1))]
for m in range(Q + 1):
    for i in range(1, L):
        G_m[m][i] = (x[i]**m / A[m][i]) * G[Q - m][i - 1]

# Вычисление стационарных характеристик

# стац. вер. что в системах не менее m треб.
PM = [[0] * L for _ in range((Q + 1))]
# стац. вер. что в системах ровно m треб.
Pm = [[0] * L for _ in range((Q + 1))]
# м.о. числа треб. в системах
s = [0] * L
# м.о. числа занятых приборов в системах
h = [0] * L
# м.о. числа треб. ожидающих в очереди систем 
b = [0] * L
# интенсивности входного потока треб. в системах
lambdas = [0] * L
# коэфф. использования систем
psy = [0] * L
# м.о. длит. пребывания треб. в системах
u = [0] * L
# м.о. длит. ожидания треб. в очереди систем
w = [0] * L
# м.о. длит. реакции сети обслуживания для систем
zita = [0] * L


for i in range(Q + 1):
    for j in range(L):
        Pm[i][j] = G_m[i][j] / G[Q][L - 1]
        summa = 0
        for k in range(m, Q + 1):
            summa += G_m[k][j]
        PM[i][j] = (1 / G[Q][L - 1]) * summa
        
        
print('\nСтационарные вероятности пребывания в системах m или более требований\n')
print(' ' * 40, 'Номер системы')
print('Кол.треб.', ' ' * 5, '1', ' ' * 7, '2', ' ' * 7, '3', ' ' * 7, '4', ' ' * 7,'5', ' ' * 7, '6', ' ' * 7, ' 7',' ' * 7, '8')
for i in range(Q + 1):
    print(i, end='\t| ')
    for j in range(L):
        print('{:07}'.format(round(PM[i][j], 5)), end=' | ')
    print('\n', '-' * 88)
        
print('#' * 100)   
        
print('\nСтационарные вероятности пребывания в системах ровно m требований\n')
print(' ' * 40, 'Номер системы')
print('Кол.треб.', ' ' * 5, '1', ' ' * 7, '2', ' ' * 7, '3', ' ' * 7, '4', ' ' * 7, '5', ' ' * 7, '6', ' ' * 7, ' 7', ' ' * 7, '8')
for i in range(Q + 1):
    print(i, end='\t| ')
    for j in range(L):
        print('{:07}'.format(round(Pm[i][j], 5)), end=' | ')
    print('\n', '-' * 88)

# Перенумерация систем обслуживания в сети 
# и вернуться к пункту вычисления базовых функций 


# Вывод результатов




Стационарные вероятности пребывания в системах m или более требований

                                         Номер системы
Кол.треб.       1         2         3         4         5         6          7         8
0	| 00000.0 | 0.00555 | 0.00169 | 00.0006 | 00000.0 | 0.00046 | 
 ----------------------------------------------------------------------------------------
1	| 00000.0 | 0.00555 | 0.00169 | 00.0006 | 00000.0 | 0.00046 | 
 ----------------------------------------------------------------------------------------
2	| 00000.0 | 0.00555 | 0.00169 | 00.0006 | 00000.0 | 0.00046 | 
 ----------------------------------------------------------------------------------------
3	| 00000.0 | 0.00555 | 0.00169 | 00.0006 | 00000.0 | 0.00046 | 
 ----------------------------------------------------------------------------------------
4	| 00000.0 | 0.00555 | 0.00169 | 00.0006 | 00000.0 | 0.00046 | 
 ----------------------------------------------------------------------------------------
5	| 00000